In [1]:
import pandas as pd 
import numpy as np
import os 
import random
from dotenv import load_dotenv
load_dotenv()

os.chdir("..")
from app.entities.chat import Chat, Survey, ProductReview
from app.entities.enumerators import *




In [2]:
there_is_full_data = True

full_data_path = "./data/full_data.csv"
chat_file = "./data/rnr_chat_w_transcript.csv"
agent_trans_file = "./data/agent_transcript_topics.csv"
customer_trans_file = "./data/customer_transcript_topics.csv"
survey_file = "./data/RnR PCS Survey 2021-10-08.csv"
product_file = "./data/RnR Product Reviews 2021-10-08.csv"
topic_file = "./data/Rocks_N_Ropes_Chat_2022-06-21v2.csv"
date_dim_file = "./data/date_dim.csv"
scenarios_file = "./data/scenarios.csv"
customer_file = "./data/customers.csv"
product_customer_file = "./data/product_review_customers.csv"
agent_file = "./data/agents.csv"
products_file = "./data/products.csv"
text_builders_file = "./data/text_drivers_2.xlsx"

# read in data
chats = pd.read_csv(chat_file)
surveys = pd.read_csv(survey_file)
product_reviews = pd.read_csv(product_file) 
topics = pd.read_csv(topic_file)
agent_trans = pd.read_csv(agent_trans_file)
customer_trans = pd.read_csv(customer_trans_file)
scenarios = pd.read_csv(scenarios_file)
customers = pd.read_csv(customer_file)
agents = pd.read_csv(agent_file)
products = pd.read_csv(products_file)
product_customers = pd.read_csv(product_customer_file)
text_builders = pd.read_excel(text_builders_file,sheet_name=None)

if there_is_full_data: 
    full_data = pd.read_csv(full_data_path,sep="|")
else: 
    full_data = pd.concat([customers]*10)

    # add Contact Type
    full_data["contact_type"] = full_data.apply(lambda x : ContactType.random_by_dist(proba=[.0858,.1012,.1284,.6347,.0499]).name, axis =1)
    full_data["product_name"] = full_data.apply(lambda x : Product.random_by_dist(proba=[0.053,0.111,0.102,0.085,0.054,0.027,0.074,0.058,0.017,0.049,0.057,0.042,0.057,0.051,0.035,0.010,0.047,0.063,0.008]).name, axis =1)

    # add new id
    full_data.insert(0,"new_chat_id", range(1,1 + len(full_data)))
    full_data.shape
    full_data.to_csv("./data/full_data.csv",index=False,sep="|")

# remove index becauase I forgot index = false
agent_trans = agent_trans.iloc[:,1:]
customer_trans = customer_trans.iloc[:,1:]
chats = chats.iloc[:,1:]


In [3]:
# # get a model prepared 
# from transformers import T5Tokenizer, T5ForConditionalGeneration, AutoTokenizer
# tokenizer = AutoTokenizer.from_pretrained("t5-base")
# model = T5ForConditionalGeneration.from_pretrained("t5-base")

In [3]:
# some dumb functions 

def generate_text(task, tokenizer, model):
    # just used what is in memoroy already
    input_ids = tokenizer.encode(task, return_tensors='pt')
    greedy_output = model.generate(input_ids, num_beams=7, no_repeat_ngram_size=2, min_length=50, max_length=100)
    message = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
    return message

def build_comment(topic_item, product, contact_type, tokenizer, model): 
    topic = topic_item.Topic.values[0]
    keyword = topic_item.Keyword.values[0]
    task = f"summarize:{product} {topic} {keyword} {contact_type}"
    task = task.replace("_"," ")
    message = generate_text(task=task, tokenizer=tokenizer, model=model)
    return message

def generate_random_date(start_date,end_date):
    import datetime
    import random

    # time_between_dates = end_date - start_date
    # days_between_dates = time_between_dates.days
    random_number_of_days = random.randrange((end_date - start_date).days)
    random_date = start_date + datetime.timedelta(days=random_number_of_days, hours=random.randrange(0,24),minutes=random.randrange(0,60), seconds=random.randrange(0,60))
    return random_date.strftime("%Y-%m-%d %H:%M:%S")

# Outtage User Story
Need interactions and Surveys that map to website outtage on a particular 2-3 day span and then drop off 

In [4]:
obj = {
    "contact_type":None,
    "site_down":None,
    "sale_accept":None,
    "site_down_sentiment":None,
    "product_name":None,
    "product_family":None,
    "order_identifier":None,
    "carrier":None,
    "customer_state":None,
    "slow_ship_state":None,
    "wismo_complaint":None,
    "promo_fail_reason":None,
    "code_exception":None,
    "purchase_date":None,
    "warranty":None,
    "rma":None,
    "return_reason":None,
    "exchange_offered":None,
    "exchange_accepted":None,
    "gift":None,
    "refund":None,

}

obj_2 = {
    "day_of_week":None,
    "week_number":None,
    "m_contact_day_number":None,
    "m_contact_time_start":None,
    "m_contact_time_end":None,
    "m_contact_duration":None,
    "m_contact_messages":None,
    "m_agent_ID":None,
    "m_agent_name":None,
    "m_agent_team_ID":None,
    "m_agent_extension":None,
    "m_agent_hire_date":None,
    "m_customer_mail":None,
    "m_customer_phone":None,
    "m_customer_zip":None,
    "m_customer_gender":None,
    "m_membership_start":None,
    "m_customer_level":None,
    "m_customer_member_number":None,
}


topic_filter = ["Errors","Crashing","Website Feedback-Negative","Website-Broken Links/Pages"]
tops = topics[topics.Topic.isin(topic_filter)].copy()

# filter for site down
text_builders_survey = text_builders["site_down_cust_survey"]
text_builders_survey = text_builders_survey[text_builders_survey.type=="site_down"].copy()
text_builders_survey["min"] = text_builders_survey["survey range"].apply(lambda x: int(x[0]))
text_builders_survey["max"] = text_builders_survey["survey range"].apply(lambda x: int(x[2]))

# Create full data set based on Contact Type

## break down

- defect 8% 
- promos - 9.8%
- returns - 13.95 %
- Sales Order 63.04%
- WISMO 5.12% 


# User story 1 site down

## assumptions 
of full data:
- 10% of full data
- happens over a 2-3 day period 
- impacts a sub set of surveys
- don't care about keeping original contact type distribution - will randomly take from full data

In [5]:
text_builders_survey.head()
text_builders_survey.response.sample(n=1).values[0]

"If your site worked I wouldn't have needed to contact you, but the site keeps crashing."

In [8]:
outtage_customers = full_data.sample(frac=.1,replace=False, random_state=45)
# set site down
outtage_customers = outtage_customers.assign(site_down = 1)
outtage_customers["site_down_sentiment"] = outtage_customers.apply(lambda x : SiteDownSentiment.random_by_dist(proba=[.5,.5,0]).name, axis =1)
outtage_customers.contact_type.value_counts()

Sales_Order    3112
Return          662
Promo           525
Defect          457
WISMO           244
Name: contact_type, dtype: int64

In [9]:
import random
import dask.dataframe as dd
from dask.distributed import Client,LocalCluster

def start_client(workers,threads,use_client=True):
    if use_client:
        cluster = LocalCluster("127.0.0.1:8786", n_workers=8, threads_per_worker=8)
        client = Client(cluster)
    return client

def shutdown_dask(client):
    client.close()
    client.shutdown()


def comment_task(df,tokenizer,model): 
    df["new_comment"] = df.apply(lambda x: build_comment(tops.sample(1), x.product_name,x.contact_type,tokenizer, model),axis=1)
    return df

def comment_build(x,s_df): 
    # filter 
    filtered = s_df[(s_df["min"] <= x.survey_score) & (s_df["max"] >= x.survey_score)].copy()
    return filtered["response"].sample(n=1).values[0]

def comment_task_dumb(df,selection_df):

    """
    What was my idea here? what was I trying to accomplish? 
    1. for each df : 
        based on product and contact type 
            select random text from text builder df 

    """
    df["new_comment"] = df.apply(lambda x: comment_build(x,selection_df), axis=1)
    return df


# just grab some preseeded surveys
outtage_surveys = surveys.sample(n=(int(.2*outtage_customers.shape[0])))
outtage_surveys["survey_score"] = outtage_surveys.apply(lambda x : SurveyScore.random_by_dist(proba=[.2,.3,.1,.2,.2,0,0,0,0,0]).value, axis =1)

# create a random seelection of id base don 
matched_ids = random.sample(range(len(outtage_customers.new_chat_id)), outtage_surveys.shape[0])
outtage_surveys["new_chat_id"] = matched_ids

# merge

outtage_surveys  = outtage_surveys.merge(outtage_customers[["new_chat_id","product_name","contact_type"]], how="inner",left_on="new_chat_id",right_on="new_chat_id")
# generate comments

outtage_surveys["new_comment"] = outtage_surveys.apply(lambda x: comment_build(x,text_builders_survey), axis=1)

# # # use dask becuase slow - but still too slow
# client = start_client(workers=8, threads=8)
# dd_outtage_surveys = dd.from_pandas(outtage_surveys,npartitions=8)
# client = start_client(workers=8, threads=8)

# dd_outtage_surveys = dd_outtage_surveys.map_partitions(lambda df: comment_task_dumb(df,text_builders_survey))
# final_df = dd_outtage_surveys.compute()
# shutdown_dask(client)

In [10]:
import datetime
outtage_surveys["new_create_date"] = outtage_surveys.apply(lambda x: generate_random_date(start_date=datetime.datetime(2022,4,4),end_date=datetime.datetime(2022,4,6)), axis=1)

In [11]:
outtage_surveys.to_csv("./data/website_down.csv",index=False, sep="|")

# Exchange offer storry

## points
- 60% of returns exchange offered 
- 85% of of exchanges need to be accepted 
- survey scores need to increase by 1.2 above the average 
- no agent concentration
## steps
1. create the interactions 
2. establish the types 


In [6]:
exchange_customers = customers.sample(frac=.6 ,replace=False, random_state=45)
exchange_customers = pd.concat([exchange_customers]*5, ignore_index=True)
exchange_customers.shape

In [8]:
# create random selection of exchange surveys

,id,first_name,last_name,email,street_address,city,state,zip,phone,gender,membership_start,customer_level,member_number,Frequency
0,4989,Allyce,Rubinovici,arubinovicird@canalblog.com,126 Elmside Park,Minneapolis,Minnesota,55428.0,612-277-4093,NaN,NaN,NaN,NaN,2
1,1828,Deloris,Cove,dcovee8@ftc.gov,27 Independence Trail,New York City,New York,10160.0,212-682-0651,NaN,NaN,NaN,NaN,7
2,2479,Bamby,Chopping,bchoppingc8@about.com,6521 Sheridan Drive,El Paso,Texas,79905.0,915-906-3429,NaN,NaN,NaN,NaN,10
3,3117,Gerda,Hardacre,ghardacrea0@creativecommons.org,104 Beilfuss Junction,Valdosta,Georgia,31605.0,229-996-8396,NaN,NaN,NaN,NaN,19
4,3034,Adler,Side,aside6u@deliciousdays.com,445 Green Ridge Point,Grand Rapids,Michigan,49505.0,616-651-5970,NaN,NaN,NaN,NaN,13


In [13]:
DateClass = DateDim()